1. 转rda -> csv
2. 合并输入输出文件（设置胎数123）
3. 处理bug
4. 划分测试训练集
5. 编码
6. 缺失值处理


In [1]:
import pyreadr
import pandas as pd
import os
import numpy as np

root_path = '/vepfs-sha/xiezixun/pregnant/data'
rda_path = os.path.join(root_path, 'rda')
original_path = os.path.join(root_path, 'original_csv_test')
input_set_123_path = os.path.join(root_path, 'input_set_123_test')

# 设置完整显示所有行列
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# 1. 转rda -> csv
def convert_rda_csv(file_path, name, target_path):
    data = pyreadr.read_r(os.path.join(file_path, name))
    data = data[name[:-4]]
    df = pd.DataFrame(data)
    df.to_csv(os.path.join(target_path, name.replace('rda', 'csv')), index=False)

for name in os.listdir(rda_path):
    convert_rda_csv(rda_path, name, original_path)

# 将hight_risk表项进行one-hot编码，并赋值其危险程度
def deal_hight_risk_one_hot():
    data = pd.read_csv(os.path.join(original_path, 'highrisk.csv'), low_memory=False)
    data = data[['母亲ID', 'highrisk_name', 'riskclass_code']]
    # 对"highrisk_name"列进行One-Hot编码
    one_hot_encoded = pd.get_dummies(data['highrisk_name'], prefix='highrisk')
    # 逐列对'highrisk_name'赋值 "riskclass_code" 的值
    for column in one_hot_encoded.columns:
        one_hot_encoded[column] *= data['riskclass_code']
    data = pd.concat([data['母亲ID'], one_hot_encoded], axis=1)
    # 合并具有相同ID的编码结果
    data = data.groupby(data['母亲ID']).sum().reset_index()
    # 更改列名
    data = data.rename(columns=lambda x: x.replace(">", "大于"))
    data = data.rename(columns=lambda x: x.replace("<", "小于"))
    data = data.rename(columns=lambda x: x.replace("≤", "小于等于"))
    data = data.rename(columns=lambda x: x.replace("≥", "大于等于"))
    data.to_csv(os.path.join(original_path, 'hightrisk_onehot.csv'), index=False)
deal_hight_risk_one_hot()

/vepfs-sha/xiezixun/miniconda3/envs/pregnant/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/vepfs-sha/xiezixun/miniconda3/envs/pregnant/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)


In [4]:
# 2. 合并输入输出文件（设置胎数123）
# 合并first和mother表
df1 = pd.read_csv(os.path.join(original_path, 'mother.csv'), low_memory=False)
df2 = pd.read_csv(os.path.join(original_path, 'first.csv'), low_memory=False)
df = pd.merge(df1, df2, on='母亲ID', how='outer')

# 合并highrisk表
df_highrisk = pd.read_csv(os.path.join(original_path, 'hightrisk_onehot.csv'), low_memory=False)
df = pd.merge(df, df_highrisk, on='母亲ID', how='left')

# 合并母亲信息和孩子信息
df3 = pd.read_csv(os.path.join(original_path, 'birth_1tai.csv'), low_memory=False)
df4 = pd.read_csv(os.path.join(original_path, 'birth_2tai.csv'), low_memory=False)
df5 = pd.concat([df3, df4], ignore_index=True)
df5['分娩时间'] = df5['分娩时间'].apply(lambda x: x[:19])     # 删除 2021-10-01 20:28:01.100 时间格式的毫秒，方便统一转换
df5['分娩时间'] = pd.to_datetime(df5['分娩时间'], errors='coerce')    # 将分娩时间列转换为 datetime 类型，方便排序
# 合并的结果表：只保留有孩子有母亲的
input_df = pd.merge(df, df5, on='母亲ID', how='inner')
input_df = input_df.sort_values(['母亲ID', '分娩时间'])
# 合并孕期增重
df_wg = pd.read_csv(os.path.join(original_path, 'weightgain14_22.csv'), low_memory=False)
df_wg_mid = pd.read_csv(os.path.join(original_path, 'weightgain_mid_cleaned.csv'), low_memory=False)
df_wg = df_wg[['母亲ID','weightgain']]
df_wg_mid = df_wg_mid[['母亲ID','weightgain','weightgain_mid']]
df_wg_all = pd.merge(df_wg,df_wg_mid,on='母亲ID',how='outer')
df_wg_all['weightgain'] = df_wg_all['weightgain_x'].combine_first(df_wg_all['weightgain_y'])
df_wg_all = df_wg_all[['母亲ID','weightgain','weightgain_mid']]
input_df = pd.merge(input_df,df_wg_all,on='母亲ID',how='left')
# 合并产后出血
df_PPH = pd.read_csv(os.path.join(original_path,'postbirthf14_22_clean.csv'),low_memory=False)
df_PPH = df_PPH[['母亲ID','产后出血']]
input_df = pd.merge(input_df,df_PPH,on='母亲ID',how='left')
# 添加 xth_child 列
input_df['xth_child'] = input_df.groupby('母亲ID').cumcount() + 1
# 重新排序列顺序
input_df = input_df[[input_df.columns[0]] + ['xth_child'] + list(input_df.columns[1:])]
input_df.to_csv(os.path.join(input_set_123_path, 'input.csv'), index=False)

In [14]:
# 3. bug处理
"""
3.0. 删除废列
3.1. 输出：premature的NA处理0，其余按程度递增处理，提取数据
3.2. 输入：处理下frist里的孕次，处理成>=1
3.3. 输入：统一mother表和first表的身高体重
3.4. 输入：将first表中的指标转为数字
3.5. 输入：处理bmi的相关指标
3.6. 输出：补正low_BW, macrosomia, LGA_S1, SGA_S1，将死亡畸形早产的这些数据补正为0
3.7. 输出：删除输出数据中的缺失项
3.8. 处理母亲，父亲民族
3.9. 处理异常值
3.10.将highrisk中nan值设为0
3.11.补正部分学长自己处理的数据
3.12.删除重复行
3.13.处理分娩方式
3.14.处理产后出血
"""

def del_column(df):
    # first.csv表：[sbp，dbp，height_first，weight_first]中的数据更加全面，删除['血压1', '血压1', '身长', '体重']
    del_first = ['血压1', '血压2', '体重_y', '身长_x']

    # mother.csv表：[age_mo, age_fa, height_mo, weight_mo, lmp]中的数据更加全面，删除['年龄', '母亲年龄', '父亲年龄', '身高', '体重', '末次月经']
    del_mother = ['父亲年龄', '年龄', '身高', '体重_x', '末次月经', '母亲年龄']
    rename_mother = {'健康状况_x': '母亲健康状况'}

    # birth_tai.csv表：[birth_weight, body_length，gw]中的数据更加全面，删除['体重', '身长_y', '孕周']
    del_child = ['体重', '身长_y', '孕周']
    rename_child = {'健康状况_y': '孩子健康状况'}
    # birth_tai.csv表中不需要预测的列，也直接删除
    del_output = ['儿童ID', '分娩地点', '性别',
    '婴儿死亡', '死亡原因', '窒息时间', '畸形', '新生儿筛查', 'taishu',
    'SGA_S2', 'SGA_S3', 'LGA_S3', '孩子健康状况',
    '出生时状况', '阿氏评分', '早产原因', 'fenmian_date',
    'body_length', 'birth_weight', 'gw']
    # ['分娩时间','分娩方式']
    
    # 其他废列
    del_other = ['xth_child.1', 'folic_pre', 'folic_dur']
    # ['母亲ID']

    # # 删除一些杂列，尝试效果
    # del_test = ['文化程度', '民族', '职业', '服用叶酸', '全面两孩', '单独两孩', '痛经', '婚龄', '母亲健康状况', '月经颜色', '月经量', '月经血块', '父亲民族', '母亲民族', 'edu_high_mo', 'edu_low_mo', 'nation_han_mo', 'nation_han_fa', 'work',
    # '首检日期', '孕产期', '检查孕周', '早孕反应', '病毒感染', '孕期服药', '营养', '发育', '经产妇' 
    # ]

    df = df.rename(columns={**rename_mother, **rename_child})
    df = df.drop(del_first + del_mother + del_child + del_output + del_other, axis=1)
    return df


# 3.1. premature的NA处理0，其余按程度递增处理，提取数据
def deal_premature(data):
    mapping = {'极早产（不足28周）': 3, 
               '中至晚期早产（32至37周）': 1,
               '早期早产（28至32周）': 2}
    data['premature'] = data['premature'].map(mapping).fillna(0)
    return data

# 3.2. 处理下frist里的产次和孕次，处理成>=1
def deal_chanci_yunci(data):
    # 处理产次
    data['产次'] = data['产次'].fillna(0.0)
    # 处理孕次
    data.loc[data['孕次'] == 0.0, '孕次'] = 1.0
    data['孕次'] = data['孕次'].fillna(1.0)    
    return data

# 3.3. 统一mother表和first表的身高体重
def unify_weight_height(data):
    data['height_first'] = data['height_first'].fillna(data['height_mo'])
    data['weight_first'] = data['weight_first'].fillna(data['weight_mo']) 
    data = data.drop(['height_mo', 'weight_mo'], axis=1)
    return data
    
# 3.4. 将first表中的指标转为数字
def deal_first_to_num(df):
    mapping = {'阴性': 0, '阳性': 1}
    df['弓形体'] = df['弓形体'].map(mapping)
    df['巨细胞病毒'] = df['巨细胞病毒'].map(mapping)
    df['风疹病毒'] = df['风疹病毒'].map(mapping)
    df['单纯疱疹病毒'] = df['单纯疱疹病毒'].map(mapping)
    
    
    # 处理另外复杂的三列
    # 尿蛋白,尿糖,尿酮体
    columns_to_process = ['尿蛋白', '尿糖', '尿酮体']
    keywords = ['阳', '+', '有', '微量', '自述诉异常', '≥','自诉异常']
    for column in columns_to_process:
        null_column = df[column].isnull()
        df[column] = df[column].apply(lambda x: 1.0 if (any(keyword in str(x) for keyword in keywords) or (isinstance(x, (int, float)) and x > 0)) else 0.0)
        df.loc[null_column, column] = np.nan
    return df           
    
# 3.5. 处理bmi的相关指标   
def deal_bmi(df):
    # 计算 BMI
    def calculate_bmi(row):
        if pd.isnull(row['height_first']) or pd.isnull(row['weight_first']):
            return float('NaN')
        else:
            return row['weight_first'] / ((row['height_first'] / 100) ** 2)

    # 判断是否肥胖，超重，过瘦的函数
    def is_overweight(row):
        if pd.isnull(row['bmi_mo']):
            return float('NaN')
        else:
            return 1.0 if row['bmi_mo'] > 25 and row['bmi_mo'] <= 27 else 0.0
    def is_overobesity(row):
        if pd.isnull(row['bmi_mo']):
            return float('NaN')
        else:
            return 1.0 if row['bmi_mo'] > 27 else 0.0    
    def is_thin(row):
        if pd.isnull(row['bmi_mo']):
            return float('NaN')
        else:
            return 1.0 if row['bmi_mo'] < 18.5 else 0.0
    # 计算
    df['bmi_mo'] = df.apply(calculate_bmi, axis=1)
    df['ob_mo'] = df.apply(is_overobesity, axis=1)
    df['thin_mo'] = df.apply(is_thin, axis=1)
    df['ow_mo'] = df.apply(is_overweight, axis=1)
    return df

# 3.6. 输出：补正low_BW, macrosomia, LGA_S1, SGA_S1，将死亡畸形早产的这些数据补正为均值
# 后续考虑比如均值等补正方式
def correct_death_child(df):
    bw_nan_df = df[((df['SGA_S1'].isna()) | df['LGA_S1'].isna() | df['macrosomia'].isna()) & ((df['death_7days'] == 1) | (df['foetal_death'] == 1) | (df['stillbirth'] == 1) | (df['malformation'] == 1) | (df['premature'] == 1) | (df['premature'] == 2) | (df['premature'] == 3))]
    # print(len(bw_nan_df))
    boolean_indexer = bw_nan_df.index
    # df.loc[boolean_indexer, 'birth_weight'] = -1
    # df.loc[boolean_indexer, 'body_length'] = -1
    df.loc[boolean_indexer, 'low_BW'] = 0
    df.loc[boolean_indexer, 'macrosomia'] = 0
    df.loc[boolean_indexer, 'SGA_S1'] = 0
    df.loc[boolean_indexer, 'LGA_S1'] = 0
    return df
    
# 3.7. 输出：删除输出数据中的缺失项
def del_miss_output(df):
    df = df.dropna(subset=['SGA_S1', 'LGA_S1', 'low_BW', 'macrosomia'])
    return df

# 3.8. 处理母亲，父亲民族
def correct_nation(df):
    nation = [
        "汉族",
        "蒙古族",
        "回族",
        "藏族",
        "维吾尔族",
        "苗族",
        "彝族",
        "壮族",
        "布依族",
        "朝鲜族",
        "满族",
        "侗族",
        "瑶族",
        "白族",
        "土家族",
        "哈尼族",
        "哈萨克族",
        "傣族",
        "黎族",
        "傈僳族",
        "佤族",
        "畲族",
        "高山族",
        "拉祜族",
        "水族",
        "东乡族",
        "纳西族",
        "景颇族",
        "柯尔克孜族",
        "土族",
        "达斡尔族",
        "仫佬族",
        "羌族",
        "布朗族",
        "撒拉族",
        "毛南族",
        "仡佬族",
        "锡伯族",
        "阿昌族",
        "普米族",
        "塔吉克族",
        "怒族",
        "乌孜别克族",
        "俄罗斯族",
        "鄂温克族",
        "德昂族",
        "保安族",
        "裕固族",
        "京族",
        "塔塔尔族",
        "独龙族",
        "鄂伦春族",
        "赫哲族",
        "门巴族",
        "珞巴族",
        "基诺族"
    ]
    df['母亲民族'] = df['母亲民族'].fillna(df['民族'])
    df['母亲民族'] = df['母亲民族'].replace('/', np.nan)
    df['母亲民族'] = df['母亲民族'].replace('亻革家人', '革家人')
    df['母亲民族'] = df['母亲民族'].apply(lambda x: x if x in nation or x is np.nan else '其他')
    df = df.drop('民族', axis=1)
    df['父亲民族'] = df['父亲民族'].replace('/', np.nan)
    df['父亲民族'] = df['父亲民族'].replace('亻革家人', '革家人')
    df['父亲民族'] = df['父亲民族'].apply(lambda x: x if x in nation or x is np.nan else '其他')
    return df

# 3.9 处理异常值
def Handle_outliers(df):
    df[['x', 'y']] = df['检查孕周'].str.split('+', expand=True)
    df['检查孕周'] = pd.to_numeric(df['x']) * 7 + pd.to_numeric(df['y'])
    df = df.drop(['x', 'y'], axis=1)
    df.loc[(df['检查孕周'] < 0) | (df['检查孕周'] > 294), '检查孕周'] = np.nan

    df.loc[(df['血糖'] < 3) | (df['血糖'] > 20), '血糖'] = np.nan
    df.loc[(df['血红蛋白'] < 80) | (df['血红蛋白'] > 250), '血红蛋白'] = np.nan
    df.loc[(df['血清谷丙转氨酶'] < 0),  '血清谷丙转氨酶'] = np.nan
    df.loc[(df['血清谷草转氨酶'] < 0),  '血清谷草转氨酶'] = np.nan
    df.loc[(df['白蛋白'] < 15) | (df['白蛋白'] > 100), '白蛋白'] = np.nan
    df.loc[(df['总胆红素'] < 0) | (df['总胆红素'] > 100), '总胆红素'] = np.nan
    df.loc[(df['结合胆红素'] < 0) | (df['结合胆红素'] > 100), '结合胆红素'] = np.nan
    df.loc[(df['血清肌酐'] < 30) | (df['血清肌酐'] > 150), '血清肌酐'] = np.nan
    df.loc[(df['血尿素氮'] < 1) | (df['血尿素氮'] > 15), '血尿素氮'] = np.nan
    df.loc[(df['婚龄'] < 0) | (df['婚龄'] > 35), '婚龄'] = np.nan
    df.loc[(df['产次'] < 0) | (df['产次'] > 10), '产次'] = np.nan
    df.loc[(df['孕次'] < 1) | (df['孕次'] > 20), '孕次'] = np.nan
    df.loc[(df['自然流产'] < 0) | (df['自然流产'] > 10), '自然流产'] = np.nan
    df.loc[(df['人工流产'] < 0) | (df['人工流产'] > 10), '人工流产'] = np.nan
    df.loc[(df['出生缺陷儿'] < 0) | (df['出生缺陷儿'] > 5), '出生缺陷儿'] = np.nan
    df.loc[(df['新生儿死亡'] < 0) | (df['新生儿死亡'] > 5), '新生儿死亡'] = np.nan
    df.loc[(df['死胎次数'] < 0) | (df['死胎次数'] > 5), '死胎次数'] = np.nan
    df.loc[(df['死产数'] < 0) | (df['死产数'] > 5), '死产数'] = np.nan

    df['月经颜色'] = df['月经颜色'].replace('月经史-痛经', np.nan)
    del_work = ['呼吸系统疾病', '扁桃体炎', '疾病与发育', '急性呼吸道感染', '重度贫血', '急性支气管炎', '维生素A缺乏症', '腹泻', '发育迟缓']
    df['职业'] = np.where(df['职业'].isin(del_work), np.nan, df['职业'])
    return df

# 3.10.将highrisk中nan值设为0
def set_highrisk_nan_0(data):
    highrisk_cols = [col for col in data.columns if col.startswith('highrisk')]
    data[highrisk_cols] = data[highrisk_cols].fillna(0)
    return data

# 3.11.补正部分学长自己处理的数据
def correct_upperclassman_deal_english_cols(df):
    conditions = ['大专', '本科', '研究生', '博士']
    df.loc[df['文化程度'].isin(conditions), 'edu_high_mo'] = 1
    df.loc[~df['文化程度'].isin(conditions), 'edu_high_mo'] = 0
    df.loc[df['文化程度'].isna(), 'edu_high_mo'] = np.nan

    conditions = ['小学', '其他']
    df.loc[df['文化程度'].isin(conditions), 'edu_low_mo'] = 1
    df.loc[~df['文化程度'].isin(conditions), 'edu_low_mo'] = 0
    df.loc[df['文化程度'].isna(), 'edu_low_mo'] = np.nan

    df.loc[df['母亲民族'] == '汉族', 'nation_han_mo'] = 1
    df.loc[df['母亲民族'] != '汉族', 'nation_han_mo'] = 0
    df.loc[df['母亲民族'].isna(), 'nation_han_mo'] = np.nan

    df.loc[df['父亲民族'] == '汉族', 'nation_han_fa'] = 1
    df.loc[df['父亲民族'] != '汉族', 'nation_han_fa'] = 0
    df.loc[df['父亲民族'].isna(), 'nation_han_fa'] = np.nan

    df.loc[df['职业'] == '无职业', 'work'] = 0
    df.loc[df['职业'] != '无职业', 'work'] = 1
    df.loc[df['职业'].isna(), 'work'] = np.nan
    return df

# 3.12.删除重复行
def drop_duplicate_rows(df):
    useless_col = [x for x in df.columns if x not in ['母亲ID','分娩时间']]
    df = df.fillna(-9999)
    df = df.drop_duplicates(subset=useless_col)
    df = df.replace(-9999, np.nan)
    return df

# 3.13.处理分娩方式
def deal_delivery_mode(df):
    df.loc[df['分娩方式']!='剖宫产','分娩方式'] = 1
    df.loc[df['分娩方式']=='剖宫产','分娩方式'] = 0
    return df

# 3.14.处理产后出血
def deal_PPH(df):
    df.loc[df['产后出血']!='是','产后出血'] = 0
    df.loc[df['产后出血']=='是','产后出血'] = 1
    return df

data = pd.read_csv(os.path.join(input_set_123_path, 'input.csv'), low_memory=False)
data = del_column(data)
data = Handle_outliers(data)    # 需要先过滤异常值，在进行后面的
data = deal_premature(data)
data = deal_chanci_yunci(data)
data = unify_weight_height(data)
data = deal_first_to_num(data)
data = deal_bmi(data)
data = correct_death_child(data)
data = del_miss_output(data)
data = correct_nation(data)
data = set_highrisk_nan_0(data)
data = correct_upperclassman_deal_english_cols(data)
data = deal_delivery_mode(data)
data = deal_PPH(data)
data = drop_duplicate_rows(data)    # 最后删重复行
data.to_csv(os.path.join(input_set_123_path, 'clean_bug_input.csv'), index=False)

/tmp/ipykernel_26681/3150429934.py:282: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(-9999)


In [ ]:
# 4. 划分测试集训练集
# 98：1：1


In [ ]:
# 5. 编码
def encode_FE(df1, df2, cols):
    for col in cols:
        df = pd.concat([df1[col],df2[col]])
        vc = df.value_counts(dropna=True, normalize=True).to_dict()
        vc[-1] = -1
        nm = col+'_FE'
        df1[nm] = df1[col].map(vc)
        df1[nm] = df1[nm].astype('float32')
        df2[nm] = df2[col].map(vc)
        df2[nm] = df2[nm].astype('float32')
        print(nm,', ',end='')

In [ ]:
# 6. 缺失值处理

In [ ]:
# df['lmp'] = pd.to_datetime(df['lmp'], errors='coerce')
# df['末次月经'] = pd.to_datetime(df['末次月经'], errors='coerce')
tc = ['母亲健康状况', 'health_mo']
data[tc] = data[tc].fillna(-999)
unequal_rows = data[data[tc[0]] != data[tc[1]]]
print('不一样的值', len(unequal_rows))
tt = (unequal_rows[tc[1]] == -999).sum()
print(tt)
print(unequal_rows[tc + ['母亲ID']].head(50))

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df